# Function Calling AWS Bedrock Converse Agent

This notebook shows you how to use our AWS Bedrock Converse agent, powered by function calling capabilities.

## Initial Setup

Let's start by importing some simple building blocks.  

The main thing we need is:
1. AWS credentials with access to Bedrock and the Claude Haiku LLM
2. a place to keep conversation history
3. a definition for tools that our agent can use.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [1]:
%pip install llama-index-llms-bedrock-converse
%pip install llama-index-embeddings-huggingface

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached aiobotocore-2.13.1-py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 138.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.2/319.2 kB 56.8 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 2.6.3
    Uninstalling networkx-2.6.3:
      Successfully uninstalled networkx-2.6.3
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.120
    Uninstalling botocore-1.34.120:
      Successfully uninstalled botocore-1.34.120
  Attempting uninstall: aioboto

In [2]:
!pip install llama-index llama-index-core llama-index-readers-file -U

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached llama_index_legacy-0.9.48-py3-none-any.whl.metadata (8.5 kB)
  Using cached llama_index_question_gen_openai-0.1.3-py3-none-any.whl.metadata (785 bytes)
  Using cached PyYAML-6.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 148.2 MB/s eta 0:00:0000:010:01
Using cached llama_index_legacy-0.9.48-py3-none-any.whl (2.0 MB)
Using cached llama_index_question_gen_openai-0.1.3-py3-none-any.whl (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 109.8 MB/s eta 0:00:00
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 26.2 MB/s e

In [1]:
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.tools import FunctionTool

import nest_asyncio

nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [3]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

Make sure to set your AWS credentials, either the `profile_name` or the keys below.

In [5]:
llm = BedrockConverse(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    # NOTE replace with your own AWS credentials
    #aws_access_key_id="AWS Access Key ID to use",
    #aws_secret_access_key="AWS Secret Access Key to use",
    #aws_session_token="AWS Session Token to use",
    region_name="us-east-1",
)

## Initialize AWS Bedrock Converse Agent

Here we initialize a simple AWS Bedrock Converse agent with calculator functions.

In [6]:
from llama_index.core.agent import FunctionCallingAgentWorker

agent_worker = FunctionCallingAgentWorker.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = agent_worker.as_agent()

### Chat

In [7]:
response = agent.chat("What is (121 + 2) * 5?")

Added user message to memory: What is (121 + 2) * 5?
=== LLM Response ===
Okay, let's break this down step-by-step:
=== Calling Function ===
Calling function: add with args: {"a": 121, "b": 2}
=== Function Output ===
123
=== LLM Response ===
So (121 + 2) = 123
=== Calling Function ===
Calling function: multiply with args: {"a": 123, "b": 5}
=== Function Output ===
615
=== LLM Response ===
Therefore, the result of (121 + 2) * 5 is 615.


In [8]:
# inspect sources
print(str(response))
print(response.sources)

Therefore, the result of (121 + 2) * 5 is 615.
[ToolOutput(content='123', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 2}}, raw_output=123, is_error=False), ToolOutput(content='615', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 123, 'b': 5}}, raw_output=615, is_error=False)]


## AWS Bedrock Converse Agent over RAG Pipeline

Build an AWS Bedrock Converse agent over a simple 10K document. We use both HuggingFace embeddings and `BAAI/bge-small-en-v1.5` to construct the RAG pipeline, and pass it to the AWS Bedrock Converse agent as a tool.

In [9]:
!mkdir -p 'data/10k/'
!curl -o 'data/10k/uber_2021.pdf' 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1836k  100 1836k    0     0  7695k      0 --:--:-- --:--:-- --:--:-- 7716k


In [5]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.bedrock_converse import BedrockConverse

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
query_llm = BedrockConverse(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    # NOTE replace with your own AWS credentials
    #aws_access_key_id="AWS Access Key ID to use",
    #aws_secret_access_key="AWS Secret Access Key to use",
    #aws_session_token="AWS Session Token to use",
    region_name="us-east-1",
)

# load data
uber_docs = SimpleDirectoryReader(
    input_files=["./data/10k/uber_2021.pdf"]
).load_data()
# build index
uber_index = VectorStoreIndex.from_documents(
    uber_docs, embed_model=embed_model
)
uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm=query_llm)
query_engine_tool = QueryEngineTool(
    query_engine=uber_engine,
    metadata=ToolMetadata(
        name="uber_10k",
        description=(
            "Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)

In [9]:
from llama_index.core.agent import FunctionCallingAgentWorker

agent_worker = FunctionCallingAgentWorker.from_tools(
    [query_engine_tool], llm=query_llm, verbose=True
)
agent = agent_worker.as_agent()

In [7]:
response = agent.chat(
    "Tell me both the risk factors and tailwinds for Uber? Do two parallel tool calls."
)

Added user message to memory: Tell me both the risk factors and tailwinds for Uber? Do two parallel tool calls.
=== LLM Response ===
Okay, let's look at the risk factors and tailwinds for Uber using the 10-K report:
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What are the risk factors for Uber?"}
=== Function Output ===
Based on the provided context, some key risk factors for Uber include:

1. Negative media coverage and damage to brand reputation: The context mentions that Uber has previously received significant negative publicity and media coverage, which has created operational, compliance, and cultural challenges. Failure to maintain and enhance their brand and reputation could cause their business to suffer.

2. Challenges in managing growth and organizational structure: The context indicates that if Uber is unable to optimize its organizational structure or effectively manage its growth, it could adversely affect the company's financial performance 